# Grasping analysis for robotic hands

This project was to create two classes, using the Sympy library, in order to calculate (symbolically) the Grasp matrix, **G**, and the Hand Jacobian, **J**, which are important for analyzing the Grasping of a robot's hand.

In [1]:
import sympy as sp
import numpy as np

In [17]:
class Grasp_hand:
    def __init__(self, Ri,n_c,n_q,cp, ciei,zi):
        """Classe for Grasping analysis in 3D

        Args:
            Ri (sympy.Matrix): rotactional matrix
            n_c (int): number of contact points of the hand
            n_q (int): number of joints of the hand
            cp (list): lsit of vectors (sympy.Matrix) for the J matrix
            ciei (list): list of vectors (sympy.Matrix) for the G matrix
            zi (list): list of vectors (sympy.Matrix) with the z axis for the J matrix
        """
        self.rx,self.ry,self.rz = sp.symbols("r_x r_y r_z")
        self.Ri = Ri
        self.S = sp.Matrix([[0, -self.rz, self.ry],[self.rz,0,-self.rx],[-self.ry,self.rx,0]])
        self.cp = cp
        self.n_c = n_c
        self.n_q = n_q
        self.ciei = ciei
        self.zi = zi
        self.R_barra = []
        self.G_T_barra = None
        self.J_barra = None
        self.J = None
        self.G_T = None

    def get_g_t_barra(self):
        """Generate the transpose of the G matrix before the selection matrix (H)

        Returns:
            sympy.Matrix: the transpose of the G matrix.
        """
        Gi_T_barra = []
        for i in range(self.n_c):
            Spi = self.S.subs([(self.rx,self.cp[i].row(0)[0]),(self.ry,self.cp[i].row(1)[0]),(self.rz,self.cp[i].row(2)[0])])
            Ri_barra = sp.Matrix([[self.Ri[i], sp.zeros(3)],[sp.zeros(3), self.Ri[i]]])
            Pi = sp.Matrix([[sp.eye(3),sp.zeros(3)],[Spi, sp.eye(3)]])
            Gi = Ri_barra.T*Pi.T
            self.R_barra.append(Ri_barra)
            Gi_T_barra.append(sp.Matrix([[Gi.row(i).col(j)[0].simplify() for j in range(6)] for i in range(6)]))
        self.G_T_barra = sp.Matrix([[gi] for gi in Gi_T_barra])
        return self.G_T_barra

    def d_i_j(self,joint,c=0,z=0):
        if joint:
            Sc = self.S.subs([(self.rx,c.row(0)[0]),(self.ry,c.row(1)[0]),(self.rz,c.row(2)[0])])
            return -Sc*z
        else:
            return sp.Matrix([0,0,0])

    def k_i_j(self,joint,z=0):
        if joint:
            return z
        else:
            return sp.Matrix([0,0,0])

    def get_j_barra(self):
        """Generate the J matrix before the selection matrix (H)

        Returns:
            sympy.Matrix: the J matrix before the selection matrix (H).
        """
        Ji_barra = []
        for i in range(self.n_c):
            d = []
            k = []
            for j in range(self.n_q):
                if self.ciei[i][j] != 0:
                    d.append(self.d_i_j(True,c=self.ciei[i][j],z=self.zi[i][j]))
                    k.append(self.k_i_j(True,z=zi[i][j]))
                else:
                    d.append(self.d_i_j(False))
                    k.append(self.k_i_j(False))
            Zi = sp.Matrix([d,k])
            Ji = (self.R_barra[i].T)*(Zi)
            Ji_barra.append(sp.Matrix([[Ji.row(i).col(j)[0].simplify() for j in range(self.n_q)] for i in range(6)]))
        self.J_barra = sp.Matrix([[ji] for ji in Ji_barra])
        return self.J_barra

    def get_G_T(self,H):
        """Returns the transpose of the G matrix after the selection matrix (H)

        Args:
            H (sympy.Matrix): selection matrix

        Returns:
            sympy.Matrix: the transpose of the G matrix after the selection matrix (H)
        """
        self.G_T = H*self.G_T_barra
        return self.G_T

    def get_J(self,H):
        """Returns the J matrix after the selection matrix (H)

        Args:
            H (sympy.Matrix): selection matrix

        Returns:
            sympy.Matrix: the J matrix after the selection matrix (H)
        """
        self.J = H*self.J_barra
        return self.J

    def get_n_space(self,M):
        """Get the null space of a matrix M

        Args:
            M (sympy.Matrix): a matrix of the sympy

        Returns:
            sympy.Matrix: a matrix with the basis of the null space 
        """
        return sp.Matrix([[n for n in M.nullspace()]])
    
    def get_c_space(self,M):
        """Get the column space of a matrix M

        Args:
            M (sympy.Matrix): a matrix of the sympy

        Returns:
            sympy.Matrix: a matrix with the basis of the column space 
        """
        return sp.Matrix([[n for n in M.columnspace()]])
    


In [18]:
class Graps_hand_2d(Grasp_hand):
    def __init__(self, Ri, n_c, n_q, cp, ciei):
        """Classe for Grasping analysis in 2D

        Args:
            Ri (sympy.Matrix): rotactional matrix
            n_c (int): number of contact points of the hand
            n_q (int): number of joints of the hand
            cp (list): lsit of vectors (sympy.Matrix) for the J matrix
            ciei (list): list of vectors (sympy.Matrix) for the G matrix
            zi (list): list of vectors (sympy.Matrix) with the z axis for the J matrix
        """
        super().__init__(Ri, n_c, n_q, cp, ciei, zi=0)
        self.S = sp.Matrix([-self.ry, self.rx])

    def get_g_t_barra(self):
        Gi_T_barra = []
        for i in range(self.n_c):
            Spi = self.S.subs([(self.rx,self.cp[i].row(0)[0]),(self.ry,self.cp[i].row(1)[0])]).T
            Pi = sp.Matrix([[sp.eye(2), sp.zeros(2,1)],[Spi,sp.Matrix([1])]])
            Ri_barra = sp.Matrix([[self.Ri[i],sp.zeros(2,1)],[0,0,1]])
            Gi = Ri_barra.T*Pi.T
            self.R_barra.append(Ri_barra)
            Gi_T_barra.append(sp.Matrix([[Gi.row(i).col(j)[0].simplify() for j in range(3)] for i in range(3)]))
        self.G_T_barra = sp.Matrix([[gi] for gi in Gi_T_barra])
        return self.G_T_barra


    def d_i_j(self,joint,c=0):
        if joint:
            Sc = self.S.subs([(self.rx,c.row(0)[0]),(self.ry,c.row(1)[0])])
            return -Sc
        else:
            return sp.Matrix([0,0])

    def k_i_j(self,joint):
        if joint:
            return 1
        else:
            return 0
        
    def get_j_barra(self):
        Ji_barra = []
        for i in range(self.n_c):
            d = []
            k = []
            for j in range(self.n_q):
                if self.ciei[i][j] != 0:
                    d.append(self.d_i_j(True,c=self.ciei[i][j]))
                    k.append(self.k_i_j(True))
                else:
                    d.append(self.d_i_j(False))
                    k.append(self.k_i_j(False))
            Zi = sp.Matrix([d,k])
            Ji = (self.R_barra[i].T)*(Zi)
            Ji_barra.append(sp.Matrix([[Ji.row(i).col(j)[0].simplify() for j in range(self.n_q)] for i in range(3)]))
        self.J_barra = sp.Matrix([[ji] for ji in Ji_barra])
        return self.J_barra

### Some examples of aplication

### In 3D

In [20]:
theta, theta_1,theta_2, l1,l2,l3,l4,l5,l6,l7, r,ci,si = sp.symbols("theta theta_1 theta_2 l_1 l_2 l_3 l_4 l_5 l_6 l_7 r c_i s_i", real=True)
rx,ry,rz = sp.symbols("r_x r_y r_z")
R1 = sp.Matrix([[-sp.cos(theta_1),sp.sin(theta_1),0],[-sp.sin(theta_1),-sp.cos(theta_1),0],[0,0,1]])
R2 = sp.Matrix([[-sp.cos(theta_2),sp.sin(theta_2),0],[-sp.sin(theta_2),-sp.cos(theta_2),0],[0,0,1]])
Ri = [R1,R2]

In [21]:
c1p = sp.Matrix([r*sp.cos(theta_1),r*sp.sin(theta_1),0])
c2p = sp.Matrix([r*sp.cos(theta_2),r*sp.sin(theta_2),0])
cp = [c1p,c2p]
c1e1 = R1*sp.Matrix([l2,l1,0])
c1e2 = R1*sp.Matrix([l7,l3,0])
c2e3 = c2e4 = R2*sp.Matrix([l4,-l5,0])
c2e5 = R2*sp.Matrix([l6,0,0])
z1 = sp.Matrix([0,0,1])
z2 = sp.Matrix([0,0,1])
z3 = R2*sp.Matrix([0,-1,0])
z4 = R2*sp.Matrix([-np.sqrt(2)/2,-np.sqrt(2)/2,0])
z5 = sp.Matrix([0,0,1])

ciei = [[c1e1,c1e2,0,0,0], [0,0,c2e3,c2e4,c2e5]]
zi = [[z1,z2,0,0,0],[0,0,z3,z4,z5]]

In [22]:
H = sp.Matrix([[sp.eye(4),sp.zeros(4,8)],[sp.zeros(4,6),sp.eye(4),sp.zeros(4,2)]])
H

Matrix([
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]])

In [23]:
example_3 = Grasp_hand(Ri,2,5,cp,ciei,zi)
g_t_barra = example_3.get_g_t_barra()
j_barra = example_3.get_j_barra()
G_T = example_3.get_G_T(H)
J = example_3.get_J(H)

#### Grasp Matrix

In [24]:
G_T.T

Matrix([
[-cos(theta_1),  sin(theta_1),               0,             0, -cos(theta_2),  sin(theta_2),               0,             0],
[-sin(theta_1), -cos(theta_1),               0,             0, -sin(theta_2), -cos(theta_2),               0,             0],
[            0,             0,               1,             0,             0,             0,               1,             0],
[            0,             0,  r*sin(theta_1), -cos(theta_1),             0,             0,  r*sin(theta_2), -cos(theta_2)],
[            0,             0, -r*cos(theta_1), -sin(theta_1),             0,             0, -r*cos(theta_2), -sin(theta_2)],
[            0,            -r,               0,             0,             0,            -r,               0,             0]])

#### Jacobian Matrix

In [9]:
J

Matrix([
[-l_1, -l_3,   0,                                             0,   0],
[ l_2,  l_7,   0,                                             0,   0],
[   0,    0,   0,                                             0,   0],
[   0,    0,   0,                                             0,   0],
[   0,    0,   0,                                             0,   0],
[   0,    0,   0,                                             0, l_6],
[   0,    0, l_4, 0.707106781186548*l_4 + 0.707106781186548*l_5,   0],
[   0,    0,   0,                            -0.707106781186548,   0]])

### In 2D

In [26]:
l1,l2,l3,l4,l5,l6,l7,l8 = sp.symbols("l_1 l_2 l_3 l_4 l_5 l_6 l_7 l_8", real=True)
Ri = [sp.Matrix([[-0.8, -0.6], [0.6,-0.8]]), sp.Matrix([[1, 0], [0,1]])]
R1 = sp.Matrix([[-0.8, -0.6], [0.6,-0.8]])
R2 = sp.Matrix([[1, 0], [0,1]])
cp = [sp.Matrix([l7*0.8 +l6*0.6,-0.6*l7 + 0.8*l6]),sp.Matrix([-l8,0])]
ciei = [[sp.Matrix([0,-l1]),sp.Matrix([l2,0]),0,0,0,0,0], 
        [0,0,sp.Matrix([0,-l1]),sp.Matrix([0,-l3]),sp.Matrix([-l5,-l3]),sp.Matrix([-l5,-l3+l4]),sp.Matrix([0,-l3+l4])]]
c1e1 = sp.Matrix([0,-l1])
c1e2 = sp.Matrix([l2,0])
H = sp.Matrix([[sp.eye(2),sp.zeros(2,4)],[sp.zeros(2,3),sp.eye(2),sp.zeros(2,1)]])
H

Matrix([
[1, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0],
[0, 0, 0, 1, 0, 0],
[0, 0, 0, 0, 1, 0]])

In [27]:
example_2d = Graps_hand_2d(Ri,2,7,cp,ciei)
g_t_barra = example_2d.get_g_t_barra()
j_barra = example_2d.get_j_barra()
G_T = example_2d.get_G_T(H)
J = example_2d.get_J(H)

#### Grasp Matrix

In [28]:
G_T.T

Matrix([
[   -0.8,     -0.6, 1,    0],
[    0.6,     -0.8, 0,    1],
[1.0*l_6, -1.0*l_7, 0, -l_8]])

#### Jacobian Matrix

In [29]:
J

Matrix([
[0.8*l_1, -0.6*l_2,    0,    0,    0,          0,          0],
[0.6*l_1,  0.8*l_2,    0,    0,    0,          0,          0],
[      0,        0, -l_1, -l_3, -l_3, -l_3 + l_4, -l_3 + l_4],
[      0,        0,    0,    0,  l_5,        l_5,          0]])